In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [3]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 10
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [4]:
#might need to rewrite these



def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'airport':0,'shopping_mall':1,'metro_station':2,'street_pedestrian':3,'public_square':4,'street_traffic':5,'tram':6,'bus':7,'metro':8,'park':9}

def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[0].replace('logMelSpec\\','')
        Y_[i,labelRef[location]]=1
    return X_, Y_
#trainlist,validationlist,testlist=fileLists()
#X_train, Y_train = getData(trainlist)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


 data sorted


In [5]:
###### Convolutional model
def compileCRNN(cols,rows,nb_classes=1):
    model = Sequential()
    # conv1 layer
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=(cols,rows,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(64, (7, 7), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,7), strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    model.add(Flatten())
    # fc1 layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(prob_drop_hidden))
    model.add(BatchNormalization())
   
    # fc2 layer
    model.add(Dense(nb_classes, activation='softmax'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = compileCRNN(cols,rows,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 216, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 108, 64)       0         
__________

In [6]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\sceneModel200_simple', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [7]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 10
rows, cols = 431, 128  
nb_epoch = 200
pool_size = (3,3)                  # size of pooling area for max pooling
prob_drop_conv = 0.25                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer


trainlist,validationlist,testlist=fileLists()
print('lists sorted')
X_tr, Y_train = getData(trainlist)
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_train=np.expand_dims(X_tr,axis=3)
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)
print(X_train.shape, X_val.shape, X_test.shape)
print('data sorted')
model = compileCRNN(cols,rows,nb_classes=nb_classes)
#model = load_model('models\\sceneModel200.hdf5')
print('model compiled')
savemodelfilename='models\\sceneModel200_simple.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)
print(datetime.now())
print('model saved')

trainfiles:  6122
validationfiles:  2518
testfiles:  2518
lists sorted
data obtained
(6122, 128, 431, 1) (2518, 128, 431, 1) (2518, 128, 431, 1)
data sorted
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 216, 64)   

6122/6122 [==============================] - 23s 4ms/step - loss: 1.2150 - acc: 0.5547 - val_loss: 6.8449 - val_acc: 0.2923
Epoch 41/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.3401 - acc: 0.4917 - val_loss: 3.0230 - val_acc: 0.2192
Epoch 42/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.3230 - acc: 0.5181 - val_loss: 2.0770 - val_acc: 0.3944
Epoch 43/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.3197 - acc: 0.5064 - val_loss: 1.7376 - val_acc: 0.3789
Epoch 44/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.1816 - acc: 0.5647 - val_loss: 9.4599 - val_acc: 0.2160
Epoch 45/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.1720 - acc: 0.5702 - val_loss: 2.1672 - val_acc: 0.3324
Epoch 46/200
6122/6122 [==============================] - 23s 4ms/step - loss: 1.1784 - acc: 0.5701 - val_loss: 2.1629 - val_acc: 0.3467
Epoch 47/200
6122/6122 [==============================

Epoch 100/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.7052 - acc: 0.7468 - val_loss: 2.6544 - val_acc: 0.2351
Epoch 101/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.7230 - acc: 0.7434 - val_loss: 5.0746 - val_acc: 0.3884
Epoch 102/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.6936 - acc: 0.7576 - val_loss: 4.4962 - val_acc: 0.3268
Epoch 103/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.6504 - acc: 0.7679 - val_loss: 1.8164 - val_acc: 0.4897
Epoch 104/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.7768 - acc: 0.7225 - val_loss: 1.6578 - val_acc: 0.4921
Epoch 105/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.6905 - acc: 0.7558 - val_loss: 1.7338 - val_acc: 0.5099
Epoch 106/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.6547 - acc: 0.7694 - val_loss: 2.8250 - val_acc: 0.3185
Epoch 107/200
6122/6122 [=========

6122/6122 [==============================] - 23s 4ms/step - loss: 0.4506 - acc: 0.8412 - val_loss: 1.3676 - val_acc: 0.5635
Epoch 160/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.4096 - acc: 0.8571 - val_loss: 1.2141 - val_acc: 0.6140
Epoch 161/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.4458 - acc: 0.8492 - val_loss: 2.1990 - val_acc: 0.4098
Epoch 162/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.4993 - acc: 0.8293 - val_loss: 1.4843 - val_acc: 0.5258
Epoch 163/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.4793 - acc: 0.8350 - val_loss: 1.4751 - val_acc: 0.5556
Epoch 164/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.4247 - acc: 0.8623 - val_loss: 3.7097 - val_acc: 0.2685
Epoch 165/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.4219 - acc: 0.8499 - val_loss: 1.8342 - val_acc: 0.5250
Epoch 166/200
6122/6122 [=======================